<a href="https://colab.research.google.com/github/gn96311/Geon_Repository/blob/master/IDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os, re, glob
import time
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageMath
from keras.preprocessing.image import load_img, array_to_img, img_to_array
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import IPython.display as display

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
Dataset_dir = os.getcwd() + '/drive/My Drive/Colab/Data'
print(Dataset_dir)

/content/drive/My Drive/Colab/Data


# Image to Patches

In [4]:
def image_to_patches(img_pad, stride_x, stride_y, patch_shape = (32, 32), mini_batch_size = 256):
    PATCH_SHAPE = patch_shape
    H, W = img_pad.shape
    batches = None

    num_patches_x = (W - PATCH_SHAPE[1]) / stride_x + 1
    num_patches_y = (H - PATCH_SHAPE[0]) / stride_y + 1

    total_patches = int(num_patches_x * num_patches_y)
    res = int(total_patches % mini_batch_size)

    batches = np.zeros((total_patches, *PATCH_SHAPE))

    x = y = 0
    for n in range(total_patches):
        patch = np.array([img_pad[y:y+PATCH_SHAPE[1], x:x+PATCH_SHAPE[0]]])
        assert patch.shape == (1, *PATCH_SHAPE), "Shape mismatch, %s" % str(patch.shape)

        batches[n, :, :] = patch

        if x + PATCH_SHAPE[1] < W:
            x += stride_x

        elif x + PATCH_SHAPE[1] >= W and y + PATCH_SHAPE[0] < H:
            y += stride_y
            x = 0

        elif x + PATCH_SHAPE[1] >= W and y + PATCH_SHAPE[0] >= H:
            break

    if not res == 0:
        for i in range(mini_batch_size-res):
            batches = np.concatenate((batches, patch), axis=0)

    batches = batches.reshape(-1, mini_batch_size, PATCH_SHAPE[0], PATCH_SHAPE[1])

    return batches, res

In [5]:
#img = Image.open(Dataset_dir + '/Mona.jpg')
#img = np.asarray(img)
#img = img[12:1068, 12:1068, :]
#batches, res = image_to_patches(img, 16, 16)

# Save the patches into Folder

In [6]:
class Dataset_maker():
    def __init__(self, img_list, list_number, mode):
        self.images_list = img_list
        self.list_num = list_number
        self.mode = mode
        assert len(np.shape(self.images_list)) == 4, "The shape of Image list is not 4D, this shape is {}".format(len(np.shape(self.images_list)))
        
    def write_img(self):
        list_num = ("{}".format(self.list_num)).zfill(3)
        with tqdm(total = np.shape(self.images_list)[0] * (np.shape(self.images_list[0])[0]), desc = 'Loading') as pbar:
            i = 0
            for batch_number in range(np.shape(self.images_list)[0]):
                for images in self.images_list[batch_number]:
                    i += 1
                    image_number = ("{}".format(i)).zfill(5)
                    image = Image.fromarray(images)
                    image = image.convert('L')
                    image.save("/content/drive/My Drive/Colab/Data/Train_Data/First_train/{}/{}_{}.jpg".format(self.mode, list_num, image_number), 'JPEG')
                    #clean_image = images/255.0
                    #noise_image = clean_image + tf.random.poisson(clean_image.shape, 0.5)
                    #save_img(clean_image, 32, 32, self.list_num, image_number, mode = 'Noise')
                    #save_img(noise_image, 32, 32, image_number, mode = 'Noise')
                    pbar.update(1)

In [7]:
'''root_dir = Dataset_dir + "/raw_data/data/train/Noise/"
directory = os.listdir(root_dir)
i = 0
for file in directory:
    i += 1
    path = os.path.join(root_dir, file)
    img = Image.open(path)
    img = np.asarray(img)
    batches, res = image_to_patches(img, 32, 32)
    dataset_make = Dataset_maker(batches, i, 'Noise')
    dataset_make.write_img()'''

'root_dir = Dataset_dir + "/raw_data/data/train/Noise/"\ndirectory = os.listdir(root_dir)\ni = 0\nfor file in directory:\n    i += 1\n    path = os.path.join(root_dir, file)\n    img = Image.open(path)\n    img = np.asarray(img)\n    batches, res = image_to_patches(img, 32, 32)\n    dataset_make = Dataset_maker(batches, i, \'Noise\')\n    dataset_make.write_img()'

# Image Show Functions

In [8]:
def noisy_imshow(image, ax=plt):
    image = np.asarray(image)
    image = (image + 0.5) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    h = ax.imshow(image)
    ax.axis('off')
    return h

In [9]:
def clean_imshow(image, ax=plt):
    image = np.asarray(image)
    h = ax.imshow(image)
    ax.axis('off')
    return h

In [10]:
'''
x = train_set[0]
fig, axes = plt.subplots(ncols=2)
noisy_imshow(x[0], ax=axes[0])
axes[0].set_title('Noisy')
noisy_imshow(x[1], ax=axes[1])
axes[1].set_title('Clean')
print(f'image size is {x[0].shape}.')'''

"\nx = train_set[0]\nfig, axes = plt.subplots(ncols=2)\nnoisy_imshow(x[0], ax=axes[0])\naxes[0].set_title('Noisy')\nnoisy_imshow(x[1], ax=axes[1])\naxes[1].set_title('Clean')\nprint(f'image size is {x[0].shape}.')"

# Data Load Class

In [13]:
class Data_Loader():
    def __init__(self, root_dir, batch_size, mode):
        self.batch_size = batch_size
        self.mode = mode
        self.root_dir = root_dir

    def return_image(self, noise_path, clean_path):
        noise_img = tf.io.decode_image(tf.io.read_file(noise_path),channels = 1)
        noise_img = tf.cast(noise_img, tf.float32) / 255.0
        clean_img = tf.io.decode_image(tf.io.read_file(clean_path),channels = 1)
        clean_img = tf.cast(clean_img,tf.float32) / 255.0

        return noise_img, clean_img

    def return_test_image(self, noise_path, clean_path):
        noise_img = tf.io.decode_image(tf.io.read_file(noise_path),channels = 1)
        noise_img = tf.cast(noise_img, tf.float32) / 255.0
        clean_img = tf.io.decode_image(tf.io.read_file(clean_path),channels = 1)
        clean_img = tf.cast(clean_img,tf.float32) / 255.0

        return noise_img, clean_img
    
    def Make_Dataset(self):
        xy_list = ['Noise', 'Clean']
        if self.mode == 'train':
            for path in xy_list:
                if path == 'Noise':
                    noise_dir = os.path.join(self.root_dir, self.mode, path)
                    noise_train_path = sorted(glob(noise_dir + '/*.jpg'))
                elif path == 'Clean':
                    clean_dir = os.path.join(self.root_dir, self.mode, path)
                    clean_train_path = sorted(glob(clean_dir + '/*.jpg'))
            dataset = tf.data.Dataset.from_tensor_slices((noise_train_path, clean_train_path))
            dataset = dataset.map(self.return_image)
            dataset = dataset.batch(self.batch_size)
            dataset = dataset.shuffle(buffer_size = len(noise_train_path))

        elif self.mode == 'test':
            for path in xy_list:
                if path == 'Noise':
                    noise_dir = os.path.join(self.root_dir, self.mode, path)
                    noise_test_path = sorted(glob(noise_dir + '/*.jpg'))
                    print(noise_test_path)
                elif path == 'Clean':
                    clean_dir = os.path.join(self.root_dir, self.mode, path)
                    clean_test_path = sorted(glob(clean_dir + '/*.jpg'))
            dataset = tf.data.Dataset.from_tensor_slices((noise_test_path, clean_test_path))
            dataset = dataset.map(self.return_test_image)
            dataset = dataset.batch(self.batch_size)
        
        return dataset

In [14]:
Generate_train_ds = Data_Loader('/content/drive/My Drive/Colab/Data/Dataset/', 16, mode = 'train')
data = Generate_train_ds.Make_Dataset()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


TypeError: ignored

In [ ]:
Generate_test_ds = Data_Loader('/content/drive/My Drive/Colab/Data/Dataset/', 1, mode = 'test')
test_data = Generate_test_ds.Make_Dataset()

In [ ]:
#data_y[0]: noise/clean
#data_y[0][:]: batch

In [24]:
class input_conv_layer(tf.keras.layers.Layer):
    def __init__(self):
        super(input_conv_layer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, (3,3), padding = 'same')
        self.relu = tf.keras.layers.ReLU()
        
    def call(self, x, training=False, mask=None):
        return self.relu(self.conv(x))

In [25]:
class skip_layer(tf.keras.layers.Layer):
    def __init__(self):
        super(skip_layer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(3, (3,3), padding = 'same')
        self.relu = tf.keras.layers.ReLU()
    
    def call(self, x, training = False, mask=None):
        return self.relu(self.conv(x))

In [26]:
class Multi_LSTM(tf.keras.layers.Layer):
    def __init__(self):
        super(Multi_LSTM, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (1,1), padding = 'same')
        self.conv2 = tf.keras.layers.Conv2D(32, (1,1), padding = 'same')
        self.conv3 = tf.keras.layers.Conv2D(1, (1,3), padding = 'same')
        self.flatten = tf.keras.layers.Flatten()
    
    def concat(self, x):
        # x : 256 1 32 32
        concat_list = []
        for i in range(tf.shape(x)[3]):
            flat = tf.expand_dims(self.flatten(x[:,:,:,i]), axis = 1) # 256 1 32
            inputs = tf.shape(x)[1] * tf.shape(x)[2]
            mini_LSTM = tf.expand_dims(tf.keras.layers.GRU(inputs)(flat), -1) # 256 32 1
            concat_list.append(mini_LSTM)
        LSTM = tf.keras.layers.concatenate(concat_list, axis = -1) #256 32 32

        return LSTM
        
    def call(self, x, training = False, mask = None):
        h = self.conv3(x) # 256 32 32 1
        rot_0 = (tf.expand_dims(h, axis=1))[:,:,:,:,-1] #256 1 32 32 
        rot_90 = (tf.expand_dims(tf.image.rot90(h, k=1), axis=1))[:,:,:,:,-1]
        rot_180 = (tf.expand_dims(tf.image.rot90(h, k=2), axis=1))[:,:,:,:,-1]
        rot_270 = (tf.expand_dims(tf.image.rot90(h, k=3), axis=1))[:,:,:,:,-1]

        LSTM_0 = tf.expand_dims(self.concat(rot_0), axis = -1) #256 32 32 1
        LSTM_90 = tf.expand_dims(self.concat(rot_90), axis = -1)
        LSTM_180 = tf.expand_dims(self.concat(rot_180), axis = -1)
        LSTM_270 = tf.expand_dims(self.concat(rot_270), axis = -1)

        h_concat = tf.keras.layers.concatenate([LSTM_0, LSTM_90, LSTM_180, LSTM_270], axis = -1)
        h_concat = self.conv1(h_concat)

        return h_concat

In [27]:
class DeN_Model(tf.keras.Model):
    def __init__(self, channel = 64, DCNN_Layer = 15):
        super(DeN_Model, self).__init__()
        self.input_conv = [input_conv_layer() for _ in range(DCNN_Layer)]
        self.skip = [skip_layer() for _ in range(DCNN_Layer)]
        self.lstm_list1 = Multi_LSTM()
        self.lstm_list2 = Multi_LSTM()
        self.lstm_list3 = Multi_LSTM()
        self.next_skip1 = skip_layer()
        self.next_skip2 = skip_layer()
        self.next_skip3 = skip_layer()
        self.conv_1 = tf.keras.layers.Conv2D(1, (3,3), padding = 'same')
    
    def call(self, x, training = False, mask = None):
        print('Epoch Start')
        input_img = x
        h_return = 0
        for layer in range(15):
            h = self.input_conv[layer](x, training) #256 32 32 64
            h_skip = self.skip[layer](h, training) # 256 32 32 3
            h_skip = self.conv_1(h_skip) #256 32 32 1
            h_return = h_return + h_skip # 256 32 32 1
        
        #identity_h = tf.keras.layers.ReLU(tf.identity(h))
        start_time = time.time()
        print('LSTM Start time: {}'.format(start_time))

        h = self.lstm_list1(h) # 256 32 32 64
        h_skip = self.next_skip1(h, training) #256 32 32 3
        h_skip = self.conv_1(h_skip) #256 32 32 1
        h_return = h_return + h_skip

        print('Duration: {}'.format(time.time() - start_time))

        '''
        h = self.lstm_list2(h)
        h_skip = self.next_skip2(h, training)
        h_skip = self.conv_1(h_skip)
        h_return = h_return + h_skip
        
        h = self.lstm_list3(h)
        h_skip = self.next_skip3(h, training)
        h_skip = self.conv_1(h_skip) #256 32 32 1

        h_return = h_return + h_skip
        '''

        output = tf.keras.layers.subtract([input_img, h_return])

        return output

In [28]:
initial_learning_rate = 1e-3
#Ir_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,decay_steps = 16,decay_rate = 0.5,staircase = True)

optimizer = tf.optimizers.Adam(initial_learning_rate)
Model = DeN_Model()


Model.compile(optimizer = optimizer, loss = 'mae')

In [30]:
Model.load_weights('/content/drive/My Drive/Colab/my_checkpoint/007/')

In [32]:
checkpoint_path = '/content/drive/My Drive/Colab/my_checkpoint/007/'

checkpoint_dir = os.path.dirname(checkpoint_path)


callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = False,
    save_freq = 16)

In [ ]:
Model.fit(data, steps_per_epoch=72, epochs = 10, callbacks = [callback])

Epoch 1/10


In [ ]:
for noise, clean in test_data:
    denoised = Model(noise)

In [ ]:
ns, cl = next(iter(test_data))
img = np.concatenate([ns[0], cl[0]], axis = 1)
plt.imshow(img)

In [ ]:
np.max(denoised[0])

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(np.concatenate([im, cl_im, denoised[0]], axis = 1))

In [ ]:
print('Complete')

In [ ]:
#Model_R.save_weights('/content/drive/My Drive/Colab/my_checkpoint/002/Red/')
#Model_G.save_weights('/content/drive/My Drive/Colab/my_checkpoint/002/Green/')
#Model_B.save_weights('/content/drive/My Drive/Colab/my_checkpoint/002/Blue/')